In [6]:
import re
import import_ipynb
import hi2en_transliterator as transliterator
import clir_files as files
import googletrans
googletrans.__version__ # must be '4.0.0-rc.1'


importing Jupyter notebook from clir_files.ipynb


'4.0.0-rc.1'

In [9]:
from googletrans import Translator
translator = Translator()
hi = translator.translate('how do people die from the coronavirus', src='en', dest="hi")
print(hi)



Translated(src=en, dest=hi, text=लोग कोरोनवायरस से कैसे मरते हैं, pronunciation=log koronavaayaras se kaise marate hain, extra_data="{'confiden...")


In [11]:
from gensim.models import KeyedVectors
from gensim import models
import gensim

hin2eng = gensim.models.KeyedVectors.load_word2vec_format('embeddings/hi_mapped.emb')
eng2hin = gensim.models.KeyedVectors.load_word2vec_format('embeddings/en_mapped.emb')

for word in ['लाल', 'गैरी', 'मकान', 'मंदिर', 'सागरतट', 'गाना', 'सेब']:
    print ("%s %s" % (word, eng2hin.similar_by_vector(hin2eng[word])[0]))

लाल ('red', 0.6013444066047668)
गैरी ('mcnaney', 0.5269891619682312)
मकान ('cottage', 0.5818758606910706)
मंदिर ('temple', 0.6957460045814514)
सागरतट ('coastline', 0.5641415119171143)
गाना ('song', 0.7094157934188843)
सेब ('cherries', 0.4921698272228241)


In [12]:
eng2hin.similar_by_vector(hin2eng["गाना"])[0:25]

[('song', 0.7094157934188843),
 ('sing', 0.6309351325035095),
 ('songs', 0.6263952255249023),
 ('tune', 0.6037591099739075),
 ('singing', 0.5932566523551941),
 ('singin', 0.590872585773468),
 ('album', 0.5890897512435913),
 ('sings', 0.5883166790008545),
 ('duet', 0.5788858532905579),
 ('rendition', 0.5718907713890076)]

In [5]:
transliterator.transliterate("कोरोनावाइरस")

'koronavaers'

In [16]:
hi_emb = gensim.models.KeyedVectors.load_word2vec_format("embeddings/wiki.hi.align.vec")

In [17]:
def query_expansion(words, emb):
    expanded_words = []
    for word in words:
        similar_words = emb.similar_by_vector(word)[:3]
        for sw in similar_words:
            if sw[0] not in words:
                expanded_words.append(sw[0])
    return words + expanded_words

In [18]:
query_expansion(["मकान"], hi_emb)

['मकान', 'मकानी', 'मकानो', 'मकानों']

In [22]:
def emb_hi2en(word, num_synonyms=3):
    words = eng2hin.similar_by_vector(hin2eng[word])[:num_synonyms]
    return [word[0] for word in words]

print(emb_hi2en("कोरोनावाइरस"))
print(emb_hi2en("कोरोनावाइरस", num_synonyms=1))

['coxsackievirus', 'polerovirus', 'erbovirus']
['coxsackievirus']


In [25]:
def read_file(file_name):
    """
    Reads file.
    Parameters:
        file_name (str): The filename for file to be read.
    Returns:
        content (str): contents of the file.
    """
    with open(file_name) as file:
        content = file.read()
        return content

In [26]:
def capture_and_remove_doc_id(doc):
    """
    Utilize regex to capture document id and remove document id from the text, so that it is not processed.
    Parameters:
        doc (str): A string.
    Returns:
        doc_id (str): The document id.
        modified_doc (str): Document with the pattern <p id = xx> removed.
    """

    pattern = r"<Q ID=\d+>"
    doc_id = re.findall(r"\d+", doc)
    modified_doc = re.sub(pattern, "", doc).strip("\n")
    return doc_id[0], (re.sub(" +", " ", modified_doc)).strip()

assert capture_and_remove_doc_id("99 bottles on the wall")[1] == "99 bottles on the wall"
assert capture_and_remove_doc_id("<Q ID=2> 99 bottles on the wall")[0][0] == "2"
assert capture_and_remove_doc_id("<Q ID=2> 99 bottles on the wall")[1] == "99 bottles on the wall"

In [27]:
def convert_queries_from_src_to_dest_lang(infile, outfile, src, dest, conv_type):
    """
    Given the query filename, process and tokenize query terms in the exact same manner as the documents.
    Attributes:
        filename (str): Filename for the query file.
    Returns:
        queries (dict(int, list[str])): Return a dict for which the key is an int value representing query_id and 
        value is a list of strings representing the terms for that query.
    """
    content = read_file(infile)
    content = re.split("</Q>", content)
    queries_processed = 0
    queries = {}
    translator = Translator()
    
    with open(outfile, "a") as of:
        for query in content:
            validation_check = re.findall(r"\d+", query)
            try:
                validation_check[0]
                query_id, query = capture_and_remove_doc_id(query)
                if conv_type == "gt":
                    en_query = translator.translate(query, src=src, dest=dest).text
                elif conv_type == "emb":
                    hi_query = query.split()
                    en_query = ""
                    for word in hi_query:
                        try:
                            en_query += f" {emb_hi2en(word, num_synonyms=1)}"
                        except KeyError:
                            en_query += f" {transliterator.transliterate(word)}"
                elif conv_type == "emb_synset":
                    hi_query = query.split()
                    try:
                        hi_query = query_expansion(hi_query, hi_emb)
                    except KeyError:
                        pass
                    en_query = ""
                    for word in hi_query:
                        try:
                            for w in emb_hi2en(word):
                                en_query+= f" {w}"
                        except KeyError:
                            en_query += f" {transliterator.transliterate(word)}"   
                elif conv_type == "emb_no_oov":
                    hi_query = query.split()
                    en_query = ""
                    for word in hi_query:
                        try:
                            for w in emb_hi2en(word):
                                en_query+= f" {w}"
                        except KeyError:
                            pass
                elif conv_type == "gt_emb_synset_translit":
                    en_query = translator.translate(query, src=src, dest=dest).text
                    hi_query = query.split()
                    try:
                        hi_query = query_expansion(hi_query, hi_emb)
                    except KeyError:
                        pass
                    for word in hi_query:
                        try:
                            translation = emb_hi2en(word)
                            for w in translation:
                                if w not in en_query:
                                    en_query+= f" {w}"
                        except KeyError:
                            transliteration = transliterator.transliterate(word)
                            if transliteration not in en_query:
                                en_query += f" {transliteration}"
                of.write(f"<Q ID={query_id}>\n{en_query.strip()}\n</Q>\n\n")
                queries_processed += 1
            except IndexError:
                pass 
    print(f"translated {queries_processed} queries")

In [28]:
# keywords
infile = files.cord19_topics_keyword_HINDI
outfiles = [[files.cord19_topics_keyword_HIN2ENG_GT, "gt"], 
            [files.cord19_topics_keyword_HIN2ENG_EMB, "emb"], [files.cord19_topics_keyword_HIN2ENG_EMB_SYNSET, "emb_synset"],
           [files.cord19_topics_keyword_HIN2ENG_EMB_NO_OOV, "emb_no_oov"], 
            [files.cord19_topics_keyword_HIN2ENG_GT_EMB_SYNSET_TRANSLIT, "gt_emb_synset_translit"]]
for ofile in outfiles:
    print(ofile)
    convert_queries_from_src_to_dest_lang(infile, ofile[0], src="hi", dest="en", conv_type=ofile[1])

['data/hi/parsed/cord19.topics.keyword.hi2eng.gt.txt', 'gt']
translated 50 queries
['data/hi/parsed/cord19.topics.keyword.hi2eng.emb.txt', 'emb']
translated 50 queries
['data/hi/parsed/cord19.topics.keyword.hi2eng.emb_synset.txt', 'emb_synset']
translated 50 queries
['data/hi/parsed/cord19.topics.keyword.hi2eng.emb_no_oov.txt', 'emb_no_oov']
translated 50 queries
['data/hi/parsed/cord19.topics.keyword.hi2eng.gt.emb.synset.translit.txt', 'gt_emb_synset_translit']
translated 50 queries


In [30]:
# questions
infile = files.cord19_topics_questions_HINDI
outfiles = [[files.cord19_topics_questions_HIN2ENG_GT, "gt"], 
            [files.cord19_topics_questions_HIN2ENG_EMB, "emb"], [files.cord19_topics_questions_HIN2ENG_EMB_SYNSET, "emb_synset"],
           [files.cord19_topics_questions_HIN2ENG_EMB_NO_OOV, "emb_no_oov"],
           [files.cord19_topics_questions_HIN2ENG_GT_EMB_SYNSET_TRANSLIT, "gt_emb_synset_translit"]]

for ofile in outfiles:
    print(ofile)
    convert_queries_from_src_to_dest_lang(infile, ofile[0], src="hi", dest="en", conv_type=ofile[1])

['data/hi/parsed/cord19.topics.question.hi2eng.gt.txt', 'gt']
translated 50 queries
['data/hi/parsed/cord19.topics.question.hi2eng.emb.txt', 'emb']
translated 50 queries
['data/hi/parsed/cord19.topics.question.hi2eng.emb_synset.txt', 'emb_synset']
translated 50 queries
['data/hi/parsed/cord19.topics.question.hi2eng.emb_no_oov.txt', 'emb_no_oov']
translated 50 queries
['data/hi/parsed/cord19.topics.questions.hi2eng.gt.emb.synset.translit.txt', 'gt_emb_synset_translit']
translated 50 queries
